<a href="https://colab.research.google.com/github/dercodeKoenig/image-upscaling.net_API/blob/main/image_upscaling_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# remove default files and download test image for upscaling and background removal
!rm -r *
!wget https://image-upscaling.net/assets/images/test_image.jpg

--2025-07-30 21:06:29--  https://image-upscaling.net/assets/images/test_image.jpg
Resolving image-upscaling.net (image-upscaling.net)... 172.67.180.202, 104.21.31.237, 2606:4700:3032::ac43:b4ca, ...
Connecting to image-upscaling.net (image-upscaling.net)|172.67.180.202|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /static/assets/images/test_image.jpg [following]
--2025-07-30 21:06:30--  https://image-upscaling.net/static/assets/images/test_image.jpg
Reusing existing connection to image-upscaling.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 28794 (28K) [image/jpeg]
Saving to: ‘test_image.jpg’

test_image.jpg      100%[===================>]  28.12K  --.-KB/s    in 0s      

2025-07-30 21:06:30 (107 MB/s) - ‘test_image.jpg’ saved [28794/28794]



# setting server url and client id for future requests

In [2]:
import requests
import json

serverurl = "https://image-upscaling.net"
client_id = "1234567890asdfghjklqwertzUIOPYXC"

# Cookie with a valid 32-digit client_id will always be sent with the request
cookies = {
    "client_id": client_id
}

# upscaling api

In [3]:
## uploading an image to be processed

use_face_enhance = False
scale = 4
path = "test_image.jpg"

url = "/".join([serverurl, "upscaling_upload"])

data = {}
if use_face_enhance:
        data["fx"] = ""
data["scale"] = scale

files = {
        "image": open(path, "rb")
}

response = requests.post(url, data=data, files=files, cookies=cookies)
print(response.text)

image will be upscaled by 4


In [4]:
## getting information about the images on the server

url = "/".join([serverurl, "upscaling_get_status"])
response = requests.get(url, cookies=cookies)
print(response.json())

{'pending': ['https://image-upscaling.net/download_upscaling_data/pending/test_image_2025-07-30_21:06:31.116907.jpg'], 'processed': [], 'processing': []}


In [5]:
## downloading the processed file(s)

while True:
  url = "/".join([serverurl, "upscaling_get_status"])
  response = requests.get(url, cookies=cookies).json()

  pending = response["pending"]
  processing = response["processing"]
  processed = response["processed"]
  print(pending, processing, processed)

  # download all processed images
  for url in processed:
    filename = url.split("/")[-1]

    preview = False
    delete_after_download = True
    params = {}
    if preview:
      params["preview"] = ""
    if delete_after_download:
      params["delete_after_download"] = ""

    print("downloading", filename)
    respone = requests.get(url, cookies=cookies, params = params)
    with open(filename, "wb") as f:
      f.write(respone.content)

  # if no requests are pending or processing or processed, everything is complete
  if len(pending) == 0 and len(processing) == 0 and len(processed) == 0:
    break
  else:
    import time
    time.sleep(1)



[] ['https://image-upscaling.net/download_upscaling_data/processing/test_image_2025-07-30_21:06:31.116907.jpg'] []
[] ['https://image-upscaling.net/download_upscaling_data/processing/test_image_2025-07-30_21:06:31.116907.jpg'] []
[] ['https://image-upscaling.net/download_upscaling_data/processing/test_image_2025-07-30_21:06:31.116907.jpg'] []
[] ['https://image-upscaling.net/download_upscaling_data/processing/test_image_2025-07-30_21:06:31.116907.jpg'] []
[] [] ['https://image-upscaling.net/download_upscaling_data/processed/test_image_2025-07-30_21:06:31.116907.jpg.png']
downloading test_image_2025-07-30_21:06:31.116907.jpg.png
[] [] []


# remove-background api

In [6]:
## uploading an image to be processed

path = "test_image.jpg"

url = "/".join([serverurl, "removebg_upload"])

files = {
        "image": open(path, "rb")
}

response = requests.post(url, files=files, cookies=cookies)
print(response.text)

image will be processed


In [7]:
## getting information about the images on the server

url = "/".join([serverurl, "removebg_get_status"])
response = requests.get(url, cookies=cookies)
print(response.json())

{'pending': ['https://image-upscaling.net/download_removebg_data/pending/test_image_2025-07-30_21:06:40.988159.jpg'], 'processed': [], 'processing': []}


In [8]:
## downloading the processed file(s)

while True:
  url = "/".join([serverurl, "removebg_get_status"])
  response = requests.get(url, cookies=cookies).json()

  pending = response["pending"]
  processing = response["processing"]
  processed = response["processed"]
  print(pending, processing, processed)

  # download all processed images
  for url in processed:
    filename = url.split("/")[-1]

    preview = False
    delete_after_download = True
    params = {}
    if preview:
      params["preview"] = ""
    if delete_after_download:
      params["delete_after_download"] = ""

    print("downloading", filename)
    respone = requests.get(url, cookies=cookies, params = params)
    with open(filename, "wb") as f:
      f.write(respone.content)

  # if no requests are pending or processing or processed, everything is complete
  if len(pending) == 0 and len(processing) == 0 and len(processed) == 0:
    break
  else:
    import time
    time.sleep(1)



[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30_21:06:40.988159.jpg'] []
[] ['https://image-upscaling.net/download_removebg_data/processing/test_image_2025-07-30

# text-to-speech api

In [9]:
## submit text to be processed

url = "/".join([serverurl, "tts_submit"])

data = {}
data["text"] = "This is the text to speech api"
data["voice"] = "am_michael"
data["speed"] = 1.1

response = requests.post(url, json=data, cookies=cookies)
print(response.text)

1753909616650


In [10]:
## getting information on requests

url = "/".join([serverurl, "tts_get_data"])
response = requests.get(url, cookies = cookies)
print(response.json())

[{'req_id': 1753909616650, 'result': 'https://image-upscaling.net/download_tts_result/1753909616650', 'speed': 1.1, 'status': 'processed', 'text': 'This is the text to speech api', 'voice': 'am_michael'}]


In [11]:
## downloading all results

while True:
  url = "/".join([serverurl, "tts_get_data"])
  response = requests.get(url, cookies = cookies).json()

  for element in response:
    url = element["result"] # get the download url here, (mp3 file)
    if url != "":
      print(url)

      params = {}
      params["delete_after_download"] = "" # delete the file after downloading it
      file_content = requests.get(url, params=params, cookies=cookies).content

      filename = url.split("/")[-1]+".mp3"
      with open(filename, "wb") as f:
        f.write(file_content)

  if len(response) == 0:
    break
  else:
    import time
    time.sleep(1)


https://image-upscaling.net/download_tts_result/1753909616650


In [13]:
# display the mp3 file
from IPython.display import Audio
Audio(filename)